# Figure Carbon Ecosystem Scatter

In [ ]:
# Libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

In [ ]:
# Directories
dir05 = '../paper_deficit/output/05_prep_other/'
dir06 = '../paper_deficit/output/06_eval/'

---

In [ ]:
# Colors
from palettable.cartocolors.qualitative import Safe_7
cpal = Safe_7

colors = [cpal.mpl_colors[i] for i in np.arange(0, 7)]

dict_colors = dict(urban = colors[0],
                   pasture = colors[1],
                   crop = colors[2],
                   foresth = colors[3],
                   forestn = colors[4],
                   shrub = colors[5],
                   other = colors[6])

df_colors = pd.DataFrame() \
    .assign(keys=dict_colors.keys(),
            colors=dict_colors.values(),
            labels=['Urban', 'Pasture', 'Cropland', 'Managed forest', 
                    'Natural forest', 'Shrubland', 'Other land'])

In [ ]:
# Function to prepare data for plotting 
def prep_data(df, labels, labelx, labely, halign, valign):
    categories = ['trop1', 'trop2', 'subt', 'temp', 'bore', 'pola']
    
    df = (df.assign(cat0=pd.Categorical(df.cat0, categories))
            .sort_values('cat0')
            .reset_index(drop=True)
            .assign(label=labels, labelx=labelx, labely=labely, halign=halign, valign=valign))
    
    # Remove 'pola' as it is considered inconsistent
    df = df[df.cat0 != 'pola']
    return df

In [ ]:
def plot_scatter(ax, df, forh):
    
    """ Subplot vegetation or soil"""
    
    #Facecolor
    if forh == True:
        facecolor = 'None'
    else:
        facecolor = df.colors
    # scatter plot
    ax.scatter(x = df.pot_tha, 
               y = df.act_tha,
#               s=np.sqrt(df.value_loss/np.pi)/150,
               s=df['def'] / 50000000, 
               edgecolors=df.colors, facecolors=facecolor, 
               linewidth=2, alpha=0.75)    
    # Forest
    if forh == True:
        for i in np.arange(0, len(df)):
            ax.text(df.loc[i].labelx, 
                    df.loc[i].labely, 
                    df.loc[i].label, 
                    verticalalignment=df.loc[i].valign, 
                    horizontalalignment=df.loc[i].halign)

        for i in np.arange(0, len(df)):
            ax.plot([df.loc[i].pot_tha, df.loc[i].labelx], 
                    [df.loc[i].act_tha, df.loc[i].labely], 
                    '#000000', linestyle='--', marker='')
    # Plot settings
    ax.axline((0, 0), (10, 10), linewidth=2, color='#000000')

In [ ]:
# Get data
file_scatter = 'df_ecozone_scatter_data_prim.csv'
df_scat = pd.read_csv(os.path.join(dir05, 'fig_ecozone', file_scatter))
# Assign colors
df_scat = df_scat.assign(colors=df_scat.cat1.map(dict_colors))

# Create general and forest dataframes
df_scat_veg_all = df_scat[(df_scat.cat2 == 'cveg') & (df_scat.cat0 == 'all')]
df_scat_veg_forh = df_scat[(df_scat.cat2 == 'cveg') & (df_scat.cat0 != 'all') &
                           (df_scat.cat1 == 'foresth')]

df_scat_soil_all = df_scat[(df_scat.cat2 == 'soc') & (df_scat.cat0 == 'all')]
df_scat_soil_forh = df_scat[(df_scat.cat2 == 'soc') & (df_scat.cat0 != 'all') & 
                            (df_scat.cat1 == 'foresth')]

# Define parameters for df_scat_veg_forh
veg_labels = ['Tropical rainforest ', 'Other tropical forest ',
              ' Subtropical forest', 'Temperate forest ',
              'Boreal forest ', 'Polar forest']
veg_labelx = [145, 104, 108, 120, 55, 17]
veg_labely = [168, 20, 45, 85, 70, 45]
veg_halign = ['right', 'left', 'left', 'left', 'right', 'center']
veg_valign = ['center', 'center', 'center', 'center', 'center', 'bottom']

# Process df_scat_veg_forh
df_scat_veg_forh = prep_data(df_scat_veg_forh,
                             veg_labels, 
                             veg_labelx, veg_labely, 
                             veg_halign, veg_valign)

# Define parameters for df_scat_soil_forh
soil_labels = ['Tropical rainforest ', 'Other tropical forest ',
               ' Subtropical forest', 'Temperate forest ',
               'Boreal forest ', 'Polar forest']
soil_labelx = [50, 55, 75, 66, 70, 90]
soil_labely = [65, 30, 41, 80, 90, 110]
soil_halign = ['right', 'left', 'center', 'right', 'right', 'right']
soil_valign = ['center', 'top', 'top', 'center', 'center', 'center']

# Process df_scat_soil_forh
df_scat_soil_forh = prep_data(df_scat_soil_forh, 
                              soil_labels, 
                              soil_labelx, soil_labely, 
                              soil_halign, soil_valign)

In [ ]:
# Plot
fig, (ax0, ax1) = plt.subplots(figsize=(9, 4.5), ncols=2, nrows=1, dpi=600)

# Scatter
plot_scatter(ax0, df_scat_veg_all, False)
plot_scatter(ax0, df_scat_veg_forh, True)

plot_scatter(ax1, df_scat_soil_all, False)
plot_scatter(ax1, df_scat_soil_forh, True)

# Axis labels
ax0.set_xlabel('Mean potential AGBC + BGBC densities (tC ha$^{-1}$)')
ax0.set_ylabel('Mean actual AGBC + BGBC densities (tC ha$^{-1}$)')
ax1.set_xlabel('Mean potential SOC 0-30cm densities (tC ha$^{-1}$)')
ax1.set_ylabel('Mean actual SOC 0-30cm densities (tC ha$^{-1}$)')

# Legend
legend_elements = [*[Patch(facecolor=i, edgecolor=i, label=ii) 
                     for i,ii in zip(df_colors.colors, df_colors.labels)],
                   Line2D([], [], color="#000000", marker='o', markersize=10, 
                          linestyle='None', linewidth=2, 
                          markerfacecolor='#000000', 
                          label='Size of deficit', alpha=0.5)
                   ]

ax0.legend(handles=legend_elements, loc='upper left', 
           ncol=1, frameon=False)

for i in [ax0, ax1]:
    linewidth = 2
    i.spines['right'].set_visible(False) # remove right plot boundary
    i.spines['top'].set_visible(False) # remove top plot boundary
    i.spines['left'].set_linewidth(linewidth) # make left axis thicker
    i.spines['bottom'].set_linewidth(linewidth) # make bottom axis thicker
    i.xaxis.set_tick_params(width=linewidth) # make x-axis tick marks thicker
    i.yaxis.set_tick_params(width=linewidth) # make y-axis tick marks thicker

# Plot settings
ax0.set_ylim(-10, 190)
ax0.set_xlim(-10, 190)
ax1.set_ylim(20, 95)
ax1.set_xlim(20, 95)

fig.text(0.02, 0.97, 'A', fontsize = 'xx-large', ha='center', va='center', 
         fontweight ="bold")
fig.text(0.52, 0.97, 'B', fontsize = 'xx-large', ha='center', va='center', 
         fontweight ="bold")

fig.tight_layout()

# Export
plt.savefig(os.path.join(dir06, f'pdf/figs08_carbon_ecosystem_scatter.pdf'), bbox_inches='tight', dpi=600)
plt.savefig(os.path.join(dir06, f'png/figs08_carbon_ecosystem_scatter.png'), bbox_inches='tight', dpi=600);